In [1]:
# Imports
import numpy as np
import pandas as pd
import torch 
import torch.utils.data as data_utils

#from torch.utils.tensorboard import SummaryWriter

from datetime import datetime 

In [2]:
# load in out presaved data
df = pd.read_csv("black-scholes.csv", index_col = False)
print("Loaded df with",len(df),"rows")

Loaded df with 3980025 rows


In [3]:
# Have to do a bit of tidying up here for some reason
df.reset_index(drop=True,inplace=True)
df.drop(df.columns[0],axis=1, inplace=True)

In [4]:
df

,spot,strike,rate,time,vol,price
0,10,10,0.00,0.1,0.05,0.063078
1,10,10,0.00,0.1,0.10,0.126151
2,10,10,0.00,0.1,0.15,0.189217
3,10,10,0.00,0.1,0.20,0.252271
4,10,10,0.00,0.1,0.25,0.315309
...,...,...,...,...,...,...
3980020,190,190,0.24,4.9,0.25,131.724846
3980021,190,190,0.24,4.9,0.30,132.415126
3980022,190,190,0.24,4.9,0.35,133.561491
3980023,190,190,0.24,4.9,0.40,135.125467


In [5]:
target = torch.tensor(df['price'].values).to(torch.float32)
features = torch.tensor(df.drop('price', axis = 1).values).to(torch.float32)

# normalize features a bit to remove scaling on asset price mostly
# abfix could be done at build time
features_norm = features * torch.tensor([1/200, 1/200,1,1/5,1])

# divide price by spot to give something thats a bit more regular (%age of notional)
target_norm = target / features[:,0]

data_set = data_utils.TensorDataset(features_norm, target_norm)
train, val, test  = torch.utils.data.random_split(data_set, [0.8, 0.1, 0.1])

train_loader = data_utils.DataLoader(train, batch_size=4, shuffle=True)
validation_loader = data_utils.DataLoader(val, batch_size = 4, shuffle = True)


In [6]:
test_loader = data_utils.DataLoader(test, batch_size = 1, shuffle = True)

print("Training set has",len(train_loader),"rows")
print("features",len(features),len(features_norm))
print("target",len(target), len(target_norm))

Training set has 796006 rows
features 3980025 3980025
target 3980025 3980025


In [7]:
#dataiter = iter(train)
#feature, price = next(dataiter)
#print(feature)
#print(price)

In [8]:
# define a model

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, name=None):
        super(Net, self).__init__()
        if name:
            self.name = name

        input_nodes = 5
        hidden_layer_nodes = 32
        output_nodes = 1
        self.fc1 = nn.Linear(input_nodes, hidden_layer_nodes)
        self.fc2 = nn.Linear(hidden_layer_nodes, hidden_layer_nodes)
        self.fc3 = nn.Linear(hidden_layer_nodes, hidden_layer_nodes)
        self.fc4 = nn.Linear(hidden_layer_nodes, output_nodes)
        
        # compute the total number of parameters
        total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(self.name + ': total params:', total_params)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


model = Net(name='bs_pricer')

bs_pricer: total params: 2337


In [9]:
# lets train the model

import torch.optim as optim

#criterion = nn.CrossEntropyLoss()
#criterion = nn.MSELoss()
#criterion = lambda y_pred,y : torch.mean((y_pred - y) ** 2)
loss_fn = torch.nn.MSELoss(size_average=False)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


G:\Anaconda\envs\CLEAN\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [10]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(train_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs.squeeze(), labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 100000 == 99999:
            last_loss = running_loss / 100000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
#            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss

In [11]:
# Initializing in a separate cell so we can easily add more epochs to the same run
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

writer = None

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs.squeeze(), vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss           
        model_path = 'model_32_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
  batch 100000 loss: 0.004777037981602416
  batch 200000 loss: 0.0008237914394252596
  batch 300000 loss: 0.0006179377751449581
  batch 400000 loss: 0.0005059094917071598
  batch 500000 loss: 0.00044195710534388816
  batch 600000 loss: 0.00039244722047261063
  batch 700000 loss: 0.00035141954455283127


G:\Anaconda\envs\CLEAN\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


LOSS train 0.00035141954455283127 valid 0.00030334858456626534
EPOCH 2:
  batch 100000 loss: 0.00029138407854437164
  batch 200000 loss: 0.0002723527919736213
  batch 300000 loss: 0.00025931411942784675
  batch 400000 loss: 0.00024568636165161876
  batch 500000 loss: 0.00023643012834156395
  batch 600000 loss: 0.0002237081465545731
  batch 700000 loss: 0.00021546315625397
LOSS train 0.00021546315625397 valid 0.00018858157272916287
EPOCH 3:
  batch 100000 loss: 0.0002018441293109362
  batch 200000 loss: 0.00019575269529270258
  batch 300000 loss: 0.00018979877643752872
  batch 400000 loss: 0.00018529852934377487
  batch 500000 loss: 0.00017701130617561516
  batch 600000 loss: 0.0001711914210734613
  batch 700000 loss: 0.00016726047340364302
LOSS train 0.00016726047340364302 valid 0.00016826781211420894
EPOCH 4:
  batch 100000 loss: 0.00015631404554476263
  batch 200000 loss: 0.0001515017679297982
  batch 300000 loss: 0.00014710865465555857
  batch 400000 loss: 0.00014492950402610177
  b

In [12]:
test_iter = iter(test_loader)


In [13]:
for i in range(0,10):
    f,l = next(test_iter)
    print(f)
    print(l)
    price = model(f)
    print(price)


    print("-"*10)

tensor([[0.2000, 0.3000, 0.1700, 0.8600, 0.0500]])
tensor([0.2779])
tensor([[0.2672]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.7000, 0.7500, 0.1500, 0.1000, 0.2000]])
tensor([0.0592])
tensor([[0.0617]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.9000, 0.9500, 0.0500, 0.0400, 0.3000]])
tensor([0.0351])
tensor([[0.0280]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.6500, 0.5000, 0.0500, 0.0800, 0.2000]])
tensor([0.2465])
tensor([[0.2507]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.4500, 0.3500, 0.0500, 0.8200, 0.4500]])
tensor([0.5023])
tensor([[0.5020]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.3000, 0.7000, 0.0700, 0.0600, 0.2000]])
tensor([4.8694e-16])
tensor([[0.0024]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.8000, 0.7500, 0.2400, 0.4600, 0.1500]])
tensor([0.4604])
tensor([[0.4619]], grad_fn=<AddmmBackward0>)
----------
tensor([[0.6000, 0.3000, 0.2400, 0.0200, 0.0500]])
tensor([0.5119])
tensor([[0.5263]], grad_fn=<AddmmBackward0>)
----------
tens

In [15]:
# now compare on all test data

# Set the model to evaluation mode, disabling dropout and using population
# statistics for batch normalization.
model.eval()

running_tloss=0.0

# Disable gradient computation and reduce memory consumption.
with torch.no_grad():
    for i, vdata in enumerate(test_loader):
        vinputs, vlabels = vdata
        voutputs = model(vinputs)
        tloss = loss_fn(voutputs.squeeze(), vlabels)
        running_tloss += tloss

avg_tloss = running_tloss / (i + 1)
print('LOSS TEST {} valid {}'.format(avg_loss, avg_tloss))


LOSS TEST 0.00011054298348754692 valid 3.161091808578931e-05
